In [16]:
import os
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY', 'sk-wtv')

## Define LLM

In [17]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key="wtv",
    openai_api_base="http://localhost:8000/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
)

## Create Graph

##### States

In [18]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str
    feeling: str


##### Nodes

In [19]:
from langchain_core.messages import AIMessage, HumanMessage


def nop(state: AgentState):
    pass


def query_addressing(state: AgentState):
    """Define which plant the query is best handled by"""
    last_message = state["messages"][-1]

    message = HumanMessage(
        content=f"""You are an efficient manager.
Your task is to decide which team member can best handle the user query.
Output ONLY the selected team member name and nothing else, your life depends on it.

===
Team members

Eddie: General assistant
Progmancer: Progressive rock expert

===
User query

{last_message}
"""
    )

    ai_message = llm.invoke([message])

    team_member_name = ai_message.content.replace('.', '').strip().lower()
    return team_member_name


def eddie(state: AgentState):
    return {"messages": [AIMessage("Greetings, creator")]}


def progmancer(state: AgentState):
    return {"messages": [AIMessage("Its 1 o'clock and time for lunch")]}


## Define Edge logic

In [20]:
# Either agent can decide to end
def router(state):
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "end"
    return "continue"

## Step 5 - Graph

In [21]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

workflow.add_node("nop", lambda x: None)
# workflow.add_node("query_addressing", query_addressing)
workflow.add_node("eddie", eddie)
workflow.add_node("progmancer", progmancer)

# This means that this node is the first one called
workflow.set_entry_point("nop")

workflow.add_conditional_edges(
    "nop",
    query_addressing,
    {"eddie": "eddie", "progmancer": "progmancer"},
)

workflow.add_edge("eddie", END)
workflow.add_edge("progmancer", END)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

# Run!

In [23]:
agent_state = {
    "messages": [
        HumanMessage(content="I would like to know about photosynthesis"),
    ],
}

app.invoke(
    agent_state,
    {
        "recursion_limit": 150,
    },
)

{'messages': [HumanMessage(content='I would like to know about photosynthesis'),
  'Greetings, creator'],
 'sender': None,
 'feeling': None}

In [24]:
agent_state = {
    "messages": [
        HumanMessage(content="I really like Porcupine Tree and Riverside. Can you recommend similar bands?"),
    ],
}

app.invoke(
    agent_state,
    {
        "recursion_limit": 150,
    },
)

{'messages': [HumanMessage(content='I really like Porcupine Tree and Riverside. Can you recommend similar bands?'),
  "Its 1 o'clock and time for lunch"],
 'sender': None,
 'feeling': None}